# **Unit 2 Build Week Project**

# Imports

In [2]:
# imports
%matplotlib inline

import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install pdpbox
!{sys.executable} -m pip install statistics
!{sys.executable} -m pip install category_encoders==2.*
import math
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn
import xgboost
import pdpbox
import statistics

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from category_encoders.ordinal import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

  Using cached plotly-4.9.0-py2.py3-none-any.whl (12.9 MB)
Processing c:\users\max\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56\retrying-1.3.3-py3-none-any.whl
Processing c:\users\max\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897\sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.23.2-cp38-cp38-win_amd64.whl (6.8 MB)
  Using cached joblib-0.16.0-py3-none-any.whl (300 kB)
  Using cached scipy-1.5.2-cp38-cp38-win_amd64.whl (31.4 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached seaborn-0.10.1-py3-none-any.whl (215 kB)
Processing c:\users\max\appdata\local\pip\cache\wheels\7f\7d\f5\136844ad90a5cfb60b92f6de7afadd0574042b6012a6d16e2b\pdpbox-0.2.0-py3-none-any.whl
  Using cached psutil-5.7.2-cp38-cp38-win_amd64.whl (243 kB)
Processing c:\users\max\appdata\local\pip\cache\wheels\36\4b\c7\6af97584669b756c0d60c5ff05d5fb1f533a4e4d96e5ee92b9\statistics-1.0.3.5-py3-no

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# original dataframe

basedf = pd.read_csv('vehicles.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'vehicles.csv'

In [164]:
basedf

,region,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,lat,long
0,mohave county,3495.0,2012.0,jeep,like new,4 cylinders,gas,NaN,clean,automatic,NaN,NaN,NaN,silver,az,34.4554,-114.2690
1,oregon coast,13750.0,2014.0,bmw,good,NaN,gas,76237.0,clean,automatic,rwd,NaN,sedan,grey,or,46.1837,-123.8240
2,greenville / upstate,2300.0,2001.0,dodge,excellent,6 cylinders,gas,199000.0,clean,automatic,NaN,NaN,NaN,NaN,sc,34.9352,-81.9654
3,mohave county,9000.0,2004.0,chevrolet,excellent,5 cylinders,gas,54000.0,clean,automatic,rwd,mid-size,pickup,red,az,34.4783,-114.2710
5,maine,500.0,2010.0,NaN,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,me,42.8424,-71.1077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435844,mohave county,2500.0,2005.0,ford,fair,NaN,gas,282866.0,clean,automatic,NaN,full-size,truck,white,az,35.2414,-113.9890
435845,rapid city / west SD,2700.0,2002.0,toyota,good,6 cylinders,gas,194000.0,clean,automatic,fwd,NaN,NaN,blue,sd,44.0036,-103.3590
435846,oregon coast,2450.0,2001.0,ford,good,4 cylinders,gas,130484.0,clean,automatic,rwd,compact,other,black,or,45.5272,-123.0950
435847,oregon coast,8995.0,2013.0,mazda,NaN,NaN,gas,93339.0,clean,automatic,fwd,NaN,sedan,NaN,or,45.5231,-122.5790


In [ ]:
# Dropping county because it has no filled values, other columns aren't relevant
# Replacing price values of 0 with NaN - we don't want to keep these observations
# We then drop those rows (consider imputer instead?)

basedf = basedf.drop(['county', 'url', 'region_url', 'image_url', 'description', 'id', 'model', 'vin'], axis=1)
basedf['price'] = basedf['price'].replace(0, np.nan)
basedf = basedf.dropna(axis=0, subset=['price'])

# Removing ridiculous outliers (outrageously high prices and low prices to bait responses)

basedf = basedf[(basedf['price'] >= np.percentile(basedf['price'], 1)) & 
        (basedf['price'] <= np.percentile(basedf['price'], 99.9))]

basedf = basedf.dropna()

In [ ]:
# Split train and validation set using wrangle
def wrangle(df, randomst):
    # Feature matrix/Target vector
    X = df.drop('price', axis=1)
    y = df['price']
    
    # Random split
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size=0.2,
                                                      random_state=randomst)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                      test_size=0.2,
                                                      random_state=randomst)
    
    # Return feature matrices and target vectors
    return X_train, y_train, X_val, y_val, X_test, y_test

In [68]:
# Try gradient boosted/random forest regressor

In [ ]:
def print_metrics(baseline_mae, baseline_rmse, train_mae, train_rmse, train_r2, val_mae, val_rmse, val_r2):
    print("Baseline MAE:\n\n", 
          baseline_mae)
    print("\nBaseline RMSE:\n\n", 
          baseline_rmse,
         "\n\n---")
    
    print('\nTraining MAE:\n\n', 
          train_mae)
    print('\nTraining RMSE:\n\n', 
          train_rmse)
    print('\nTraining R2 Score:\n\n', 
          train_r2)
    
    print('\nValidation MAE:\n\n', 
          val_mae)
    print('\nValidation RMSE:\n\n', 
          val_rmse)
    print('\nValidation R2 Score:\n\n', 
          val_r2,
         "\n\n---\n")
    
    return baseline_mae, baseline_rmse, train_mae, train_rmse, train_r2, val_mae, val_rmse, val_r2

def get_metrics(X_train, y_train, X_val, y_val, model): 
    
    baseline = [y_train.mean()]*len(y_train)
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    
    baseline_mae = mean_absolute_error(y_train, baseline)
    baseline_rmse = math.sqrt(mean_squared_error(y_train, baseline))
    
    train_mae = mean_absolute_error(y_train, train_pred)
    train_rmse = math.sqrt(mean_squared_error(y_train, train_pred))
    train_r2 = r2_score(y_train, train_pred)
    
    val_mae = mean_absolute_error(y_val, val_pred)
    val_rmse = math.sqrt(mean_squared_error(y_val, val_pred))
    val_r2 = r2_score(y_val, val_pred)
    
    return print_metrics(baseline_mae, baseline_rmse, train_mae, train_rmse, train_r2, val_mae, val_rmse, val_r2)
    
def model_metrics(wrangle, model):
    
    X_train, y_train, X_val, y_val, X_test, y_test = wrangle
    
    model.fit(X_train, y_train)
    
    return get_metrics(X_train, y_train, X_val, y_val, model)

def multi_state_metrics(randomst_list, basedf, model, modelstr): 
    avg_baseline_mae = 0
    avg_baseline_rmse = 0
    
    avg_train_mae = 0
    avg_train_rmse = 0
    avg_train_r2 = 0
    
    avg_val_mae = 0
    avg_val_rmse = 0
    avg_val_r2 = 0
    
    for randomst in randomst_list:
        print(">**Random State:", str(randomst) + "**\n")
        baseline_mae, baseline_rmse, train_mae, train_rmse, train_r2, val_mae, val_rmse, val_r2 = model_metrics(wrangle(basedf, randomst), model)
        avg_baseline_mae += baseline_mae
        avg_baseline_rmse += baseline_rmse
        avg_train_mae += train_mae
        avg_train_rmse += train_rmse
        avg_train_r2 += train_r2
        avg_val_mae += val_mae
        avg_val_rmse += val_rmse
        avg_val_r2 += val_r2
    avg_list = [avg_baseline_mae, avg_baseline_rmse, avg_train_mae, avg_train_rmse, avg_train_r2, avg_val_mae, avg_val_rmse, avg_val_r2]
    avg_baseline_mae, avg_baseline_rmse, avg_train_mae, avg_train_rmse, avg_train_r2, avg_val_mae, avg_val_rmse, avg_val_r2 = [x / len(randomst_list) for x in avg_list]
    print(">**" + modelstr + " Averages:**\n")
    print_metrics(avg_baseline_mae, avg_baseline_rmse, avg_train_mae, avg_train_rmse, avg_train_r2, avg_val_mae, avg_val_rmse, avg_val_r2)

In [ ]:
'''
SI_model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    XGBRegressor(n_estimators=300,
                 random_state=42)
)
'''

DropNA_lr_model = make_pipeline(
    OrdinalEncoder(),
    LinearRegressor())

DropNA_model = make_pipeline(
    OrdinalEncoder(),
    XGBRegressor(n_estimators=905,
                 max_depth=15,
                 eta = 0.009,
                 gamma = 0.8,
                 min_child_weight = 0.026,
                 subsample = 0.512,
                 random_state=42)
)


#print("# **Simple Imputer:**\n\n" +
     #"---\n")
#multi_state_metrics([42, 43, 44], basedf, SI_model, "Simple Imputer")

print("# **DropNA:**\n\n" +
     "---\n")
multi_state_metrics([42, 43, 44], basedf, DropNA_model, "DropNA")

In [ ]:
basedf = OrdinalEncoder().fit_transform(basedf)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

folds = 3
param_comb = 5

params = { 
           'n_estimators' : [904, 905, 906],
           'max_depth' : [15, 16, 17],
           'eta' : [0.009, 0.0089, 0.0091],
           'gamma' : [0.79, 0.8, 0.81],
           'min_child_weight': [0.03, 0.028, 0.026],
           'subsample' : [0.512, 0.513, 0.514]
         }

skf = KFold(n_splits=folds, shuffle = True, random_state = 42)

random_search = RandomizedSearchCV(XGBRegressor(), 
                                   param_distributions=params, 
                                   n_iter=param_comb, 
                                   scoring='r2', 
                                   n_jobs=4, 
                                   cv=skf.split(basedf.drop('price', axis=1), basedf['price']), 
                                   verbose=3, 
                                   random_state=42)

In [59]:
random_search.fit(basedf.drop('price', axis=1), basedf['price'])

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed: 12.7min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000001B4567C0CF0>,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, miss...
                                          reg_lambda=None,
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validate_parameters=None,
     

In [61]:
random_search.best_params_

{'subsample': 0.512,
 'n_estimators': 905,
 'min_child_weight': 0.026,
 'max_depth': 15,
 'gamma': 0.8,
 'eta': 0.009}

# **Simple Imputer vs DropNA (Linear Regression)**

# **Simple Imputer:**

---

>**Random State: 42**

Baseline MAE:

 8322.7882852111

Baseline RMSE:

 11265.197800282813 

---

Training MAE:

 6411.333091493481

Training RMSE:

 9534.50542528138

Training R2 Score:

 0.2836608339038512

Validation MAE:

 6433.728270422327

Validation RMSE:

 9585.784554108888

Validation R2 Score:

 0.2712349859322897 

---

>**Random State: 43**

Baseline MAE:

 8300.929264601036

Baseline RMSE:

 11216.881681425752 

---

Training MAE:

 6409.8599648985155

Training RMSE:

 9505.994981300058

Training R2 Score:

 0.2817909328453426

Validation MAE:

 6448.557446411898

Validation RMSE:

 9536.580933454277

Validation R2 Score:

 0.29242748048605327 

---

>**Random State: 44**

Baseline MAE:

 8315.141755509649

Baseline RMSE:

 11255.942419197228 

---

Training MAE:

 6408.164413941968

Training RMSE:

 9537.217559725454

Training R2 Score:

 0.2820740460401905

Validation MAE:

 6450.304517962011

Validation RMSE:

 9568.210839216683

Validation R2 Score:

 0.28127193603257294 

---

>**Simple Imputer Averages:**

Baseline MAE:

 8312.953101773928

Baseline RMSE:

 11246.00730030193 

---

Training MAE:

 6409.785823444654

Training RMSE:

 9525.905988768964

Training R2 Score:

 0.2825086042631281

Validation MAE:

 6444.196744932079

Validation RMSE:

 9563.525442259948

Validation R2 Score:

 0.28164480081697196 

---

# **DropNA:**

---

>**Random State: 42**

Baseline MAE:

 7098.589921453132

Baseline RMSE:

 9576.825579972676 

---

Training MAE:

 5297.1517348535035

Training RMSE:

 7932.524833047396

Training R2 Score:

 0.31391215529149974

Validation MAE:

 5312.335190833456

Validation RMSE:

 8048.892507290799

Validation R2 Score:

 0.3081039192211007 

---

>**Random State: 43**

Baseline MAE:

 7102.725706740333

Baseline RMSE:

 9628.318937603497 

---

Training MAE:

 5267.370326355173

Training RMSE:

 7956.782151045945

Training R2 Score:

 0.3170734458335607

Validation MAE:

 5279.785525230837

Validation RMSE:

 8042.477638648895

Validation R2 Score:

 0.29422167719662307 

---

>**Random State: 44**

Baseline MAE:

 7087.291426064146

Baseline RMSE:

 9587.993466246098 

---

Training MAE:

 5296.959062827044

Training RMSE:

 7960.010463697251

Training R2 Score:

 0.31075786593989596

Validation MAE:

 5320.046376302943

Validation RMSE:

 7985.250598033452

Validation R2 Score:

 0.31833882849540995 

---

>**DropNA Averages:**

Baseline MAE:

 7096.202351419204

Baseline RMSE:

 9597.71266127409 

---

Training MAE:

 5287.160374678574

Training RMSE:

 7949.772482596864

Training R2 Score:

 0.31391448902165214

Validation MAE:

 5304.055697455745

Validation RMSE:

 8025.540247991048

Validation R2 Score:

 0.3068881416377112 

---

# **Simple Imputer vs DropNA (XGBoost Regressor)**

# **Simple Imputer:**

---

>**Random State: 42**

Baseline MAE:

 8322.7882852111

Baseline RMSE:

 11265.197800282813 

---

Training MAE:

 3103.3831411782417

Training RMSE:

 5037.32029405106

Training R2 Score:

 0.8000499630980258

Validation MAE:

 3237.048432226411

Validation RMSE:

 5365.724695723263

Validation R2 Score:

 0.7716563388578538 

---

>**Random State: 43**

Baseline MAE:

 8300.929264601036

Baseline RMSE:

 11216.881681425752 

---

Training MAE:

 3149.6625077879144

Training RMSE:

 5110.460237383701

Training R2 Score:

 0.792424664084626

Validation MAE:

 3329.011616788695

Validation RMSE:

 5579.186958895811

Validation R2 Score:

 0.7578260699270866 

---

>**Random State: 44**

Baseline MAE:

 8315.141755509649

Baseline RMSE:

 11255.942419197228 

---

Training MAE:

 3135.251766970912

Training RMSE:

 5112.738117867638

Training R2 Score:

 0.7936790301380974

Validation MAE:

 3283.612837393329

Validation RMSE:

 5411.805301340317

Validation R2 Score:

 0.7700743318175275 

---

>**Simple Imputer Averages:**

Baseline MAE:

 8312.953101773928

Baseline RMSE:

 11246.00730030193 

---

Training MAE:

 3129.4324719790225

Training RMSE:

 5086.8395497674655

Training R2 Score:

 0.7953845524402497

Validation MAE:

 3283.224295469479

Validation RMSE:

 5452.238985319797

Validation R2 Score:

 0.7665189135341559 

---

# **DropNA:**

---

>**Random State: 42**

Baseline MAE:

 7098.589921453132

Baseline RMSE:

 9576.825579972676 

---

Training MAE:

 1981.1625732913974

Training RMSE:

 3042.3168288715074

Training R2 Score:

 0.8990826764569256

Validation MAE:

 2330.5679177604993

Validation RMSE:

 4027.5916136279047

Validation R2 Score:

 0.8267554938115754 

---

>**Random State: 43**

Baseline MAE:

 7102.725706740333

Baseline RMSE:

 9628.318937603497 

---

Training MAE:

 1978.3814270460873

Training RMSE:

 3069.767948246008

Training R2 Score:

 0.8983493502746951

Validation MAE:

 2321.37043221614

Validation RMSE:

 3918.1078378017637

Validation R2 Score:

 0.8324897599496697 

---

>**Random State: 44**

Baseline MAE:

 7087.291426064146

Baseline RMSE:

 9587.993466246098 

---

Training MAE:

 1994.7025175649912

Training RMSE:

 3087.6756237619484

Training R2 Score:

 0.8962930457943539

Validation MAE:

 2342.9819100532054

Validation RMSE:

 3985.157499557629

Validation R2 Score:

 0.8302215998833047 

---

>**DropNA Averages:**

Baseline MAE:

 7096.202351419204

Baseline RMSE:

 9597.71266127409 

---

Training MAE:

 1984.7488393008252

Training RMSE:

 3066.5868002931547

Training R2 Score:

 0.897908357508658

Validation MAE:

 2331.640086676615

Validation RMSE:

 3976.952316995766

Validation R2 Score:

 0.8298222845481833 

---